Load the trained model

In [2]:
import numpy as np
mask = np.zeros((2,600), dtype='float32')

mask[0,:433] = 1.0
mask[1,:226] = 1.0

#mask = mask[:, None]
x_dummy = np.random.random((2,600,20))
x_dummy = np.cast['float32'](x_dummy)

In [60]:
import numpy as np
from fuel.datasets import H5PYDataset

train_set = H5PYDataset('/misc/data15/reco/bhattgau/Rnn/lists/rsr/rsr_train3.hdf5',
                        which_sets=('train',), subset=slice(0,1))
handle = train_set.open()
data = train_set.get_data(handle, slice(0,1))

x = data[0]
mask = data[1]
phr_targets = data[2]
spk_targets = data[3]

In [4]:
import htkmfc
import os
import lasagne
import time
import theano
import theano.tensor as T
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import ipdb

from fuel.datasets import IndexableDataset
from fuel.schemes import ShuffledScheme
from fuel.streams import DataStream

from collections import OrderedDict
from fuel.datasets.hdf5 import H5PYDataset


# Min/max sequence length
MAX_LENGTH = 800
# Number of units in the hidden (recurrent) layer
N_HIDDEN = 100
F_DIM = 60
# Number of training sequences ain each batch
N_BATCH = 1
# Optimization learning rate
LEARNING_RATE = .001
# All gradients above this will be clipped
GRAD_CLIP = 100
# How often should we check the output?
EPOCH_SIZE = 100
# Number of epochs to train the net
NUM_EPOCHS = 10
BATCH_SIZE = 5

nSpk = 194
nPhr = 73

X = T.tensor3(name='input',dtype='float32')
Mask = T.matrix(name = 'mask', dtype='float32')

print("Building network ...")

l_in = lasagne.layers.InputLayer(shape=(None, MAX_LENGTH, F_DIM), input_var = X)
n_batch,_,_ = l_in.input_var.shape
#print lasagne.layers.get_output(l_in, inputs={l_in: X}).eval({X: x_dummy}).shape

l_mask = lasagne.layers.InputLayer(shape=(None, MAX_LENGTH), input_var = Mask)
#print lasagne.layers.get_output(l_mask, inputs={l_mask: Mask}).eval({Mask: mask}).shape

#initialize the gates
l_forward = lasagne.layers.GRULayer(l_in, N_HIDDEN, precompute_input=True, mask_input=l_mask, only_return_final=True)
l_backward = lasagne.layers.GRULayer(l_in, N_HIDDEN, precompute_input=True, mask_input=l_mask, only_return_final=True,
                                     backwards=True)

l_sum = lasagne.layers.ElemwiseSumLayer([l_forward, l_backward])

l_spk_softmax = lasagne.layers.DenseLayer(l_sum, num_units=nSpk, nonlinearity=lasagne.nonlinearities.softmax)

l_phr_softmax = lasagne.layers.DenseLayer(l_sum, num_units=nPhr, nonlinearity=lasagne.nonlinearities.softmax)

#print lasagne.layers.get_output(l_softmax, inputs={l_in: X, l_mask: Mask}).eval({X: d1, Mask: m1}).shape
#l_softmax = lasagne.layers.ReshapeLayer(l_dense, (n_batch, MAX_LENGTH, N_HIDDEN))

spk_labels = T.ivector(name='spk_labels')
phr_labels = T.ivector('phr_labels')


network_output_spk = lasagne.layers.get_output(l_spk_softmax)
network_output_phr = lasagne.layers.get_output(l_phr_softmax)

spk_cost = lasagne.objectives.categorical_crossentropy(network_output_spk, spk_labels)
phr_cost = lasagne.objectives.categorical_crossentropy(network_output_phr, phr_labels)

total_cost = spk_cost + phr_cost
mean_cost = total_cost.mean()

val_prediction_spk = lasagne.layers.get_output(l_spk_softmax, deterministic=True)
val_prediction_phr = lasagne.layers.get_output(l_phr_softmax, deterministic=True)

Building network ...


In [5]:
val_spk_cost = lasagne.objectives.categorical_crossentropy(val_prediction_spk, spk_labels)
val_phr_cost = lasagne.objectives.categorical_crossentropy(val_prediction_phr, phr_labels)

val_cost = val_spk_cost + val_phr_cost
val_mcost = val_cost.mean()

#needed for accuracy
#don't use the one hot vectors here
#The one hot vectors are needed for the categorical cross-entropy
val_acc_spk = T.mean(T.eq(T.argmax(val_prediction_spk, axis=1), spk_labels), dtype=theano.config.floatX)
val_acc_phr = T.mean(T.eq(T.argmax(val_prediction_phr, axis=1), phr_labels), dtype=theano.config.floatX)

#training accuracy
train_acc_spk = T.mean(T.eq(T.argmax(network_output_spk, axis=1), spk_labels), dtype=theano.config.floatX)
train_acc_phr = T.mean(T.eq(T.argmax(network_output_phr, axis=1), phr_labels), dtype=theano.config.floatX)

#T.argmax(network_output, axis=1).eval({X: d1, Mask: m1})

#print network_output.eval({X: d1, Mask: m1})[1][97]
#cost function
#total_cost = -(labels*T.log(network_output) + (1-labels)*T.log(1-network_output)) 
#accuracy function


#Get parameters of both encoder and decoder
all_parameters = lasagne.layers.get_all_params([l_spk_softmax,l_phr_softmax], trainable=True)

print("Trainable Model Parameters")
print("-"*40)
for param in all_parameters:
    print(param, param.get_value().shape)
print("-"*40)
#add grad clipping to avoid exploding gradients
all_grads = [T.clip(g,-3,3) for g in T.grad(mean_cost, all_parameters)]
all_grads = lasagne.updates.total_norm_constraint(all_grads,3)

updates = lasagne.updates.adam(all_grads, all_parameters, learning_rate=0.005)

train_func = theano.function([X, Mask, spk_labels, phr_labels], [mean_cost, train_acc_spk, train_acc_phr], updates=updates)

val_func = theano.function([X, Mask, spk_labels, phr_labels], [val_mcost, val_acc_spk, val_acc_phr])



train_set = H5PYDataset('/misc/data15/reco/bhattgau/Rnn/lists/rsr/rsr_train3.hdf5',
                        which_sets=('train',), subset=slice(0,2000))

valid_set = H5PYDataset('/misc/data15/reco/bhattgau/Rnn/lists/rsr/rsr_train3.hdf5',
                        which_sets=('test',), subset=slice(0,200))

num_epochs=10

trainerr=[]

print("Starting training...")
    # We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    tr_acc_spk = 0
    tr_acc_phr = 0
    
    train_batches = 0
    
    t_state = train_set.open()
    v_state = valid_set.open()

    scheme = ShuffledScheme(examples=train_set.num_examples, batch_size=256)
    scheme1 = ShuffledScheme(examples=valid_set.num_examples, batch_size=100)


    train_stream = DataStream(dataset=train_set, iteration_scheme=scheme)
    valid_stream = DataStream(dataset=valid_set, iteration_scheme=scheme1)
    
    start_time = time.time()
    
    for data in train_stream.get_epoch_iterator():
        t_data, t_mask, t_plabs,t_labs = data
        
        terr, tacc_spk, tacc_phr = train_func(t_data, t_mask, t_labs, t_plabs)
        train_err += terr
        tr_acc_spk += tacc_spk
        tr_acc_phr += tacc_phr

        train_batches += 1
        
    val_err = 0
    val_acc_spk = 0
    val_acc_phr = 0

    val_batches = 0
    
    for data in valid_stream.get_epoch_iterator():
        v_data, v_mask, v_ptars, v_tars = data

        err, acc_spk, acc_phr = val_func(v_data, v_mask , v_tars, v_ptars)
        val_err += err
        val_acc_spk += acc_spk
        val_acc_phr += acc_phr

        val_batches += 1
        
    trainerr.append(train_err/train_batches)
    
    train_set.close(state=t_state)
    valid_set.close(state=v_state)
        
# Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
    epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  training accuracy (speaker):\t\t{:.2f} %".format(
        tr_acc_spk / train_batches * 100))
    print("  training accuracy (phrase):\t\t{:.2f} %".format(
        tr_acc_phr / train_batches * 100))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy (speaker):\t\t{:.2f} %".format(
        val_acc_spk / val_batches * 100))
    print("  validation accuracy (phrase):\t\t{:.2f} %".format(
        val_acc_phr / val_batches * 100))

Trainable Model Parameters
----------------------------------------
(W_in_to_updategate, (60, 100))
(W_hid_to_updategate, (100, 100))
(b_updategate, (100,))
(W_in_to_resetgate, (60, 100))
(W_hid_to_resetgate, (100, 100))
(b_resetgate, (100,))
(W_in_to_hidden_update, (60, 100))
(W_hid_to_hidden_update, (100, 100))
(b_hidden_update, (100,))
(W_in_to_updategate, (60, 100))
(W_hid_to_updategate, (100, 100))
(b_updategate, (100,))
(W_in_to_resetgate, (60, 100))
(W_hid_to_resetgate, (100, 100))
(b_resetgate, (100,))
(W_in_to_hidden_update, (60, 100))
(W_hid_to_hidden_update, (100, 100))
(b_hidden_update, (100,))
(W, (100, 194))
(b, (194,))
(W, (100, 73))
(b, (73,))
----------------------------------------
Starting training...
Epoch 1 of 10 took 30.935s
  training loss:		9.550815
  training accuracy (speaker):		0.83 %
  training accuracy (phrase):		3.59 %
  validation loss:		9.157307
  validation accuracy (speaker):		0.50 %
  validation accuracy:		10.00 %
Epoch 2 of 10 took 33.334s
  training

In [ ]:
# %load ../speaker_softmax.py

import htkmfc
import os
import lasagne
import time
import theano
import theano.tensor as T
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import cPickle
import sys

from fuel.datasets import IndexableDataset
from fuel.schemes import ShuffledScheme, SequentialScheme
from fuel.streams import DataStream

from collections import OrderedDict
import h5py

from fuel.datasets import H5PYDataset
from fuel.converters.base import fill_hdf5_file
from utils import load_data

cFile = sys.argv[1]

f=open(cFile)
lines = f.readlines()
lines = [l.strip() for l in lines]
values = [v.split()[1] for v in lines]


N_HIDDEN = int(values[0])
F_DIM = int(values[1])
LEARNING_RATE = np.cast['float32'](values[2])
NUM_EPOCHS = int(values[3])
NUM_SPEAKERS = int(values[4])
MODEL_NAME = values[5]
SAVE_PATH = values[6]
#data
TRAIN_FILE = values[7]
VALID_FILE = values[8]


# Min/max sequence length
MAX_LENGTH = 800

def build_rnn(net_input=None, mask_input=None):

    print("Building network ...")

    l_in = lasagne.layers.InputLayer(shape=(None, MAX_LENGTH, F_DIM), input_var = X)
    n_batch,_,_ = l_in.input_var.shape
    #print lasagne.layers.get_output(l_in, inputs={l_in: X}).eval({X: x_dummy}).shape

    l_mask = lasagne.layers.InputLayer(shape=(None, MAX_LENGTH), input_var = Mask)
    #print lasagne.layers.get_output(l_mask, inputs={l_mask: Mask}).eval({Mask: mask}).shape

    #initialize the gates
    l_forward = lasagne.layers.GRULayer(l_in, N_HIDDEN, precompute_input=True, mask_input=l_mask, only_return_final=True)
    l_backward = lasagne.layers.GRULayer(l_in, N_HIDDEN, precompute_input=True, mask_input=l_mask, only_return_final=True,
                                         backwards=True)

    l_sum = lasagne.layers.ElemwiseSumLayer([l_forward, l_backward])

    l_spk_softmax = lasagne.layers.DenseLayer(l_sum, num_units=nSpk, nonlinearity=lasagne.nonlinearities.softmax)

    l_phr_softmax = lasagne.layers.DenseLayer(l_sum, num_units=nPhr, nonlinearity=lasagne.nonlinearities.softmax)
    
    return l_spk_softmax, l_phr_softmax


#print lasagne.layers.get_output(l_softmax, inputs={l_in: X, l_mask: Mask}).eval({X: d1, Mask: m1}).shape
#l_softmax = lasagne.layers.ReshapeLayer(l_dense, (n_batch, MAX_LENGTH, N_HIDDEN))

def main():
        
    X = T.tensor3(name='input',dtype='float32')
    MASK = T.matrix(name = 'mask', dtype='float32')
    SPK_LABELS = T.ivector(name='spk_labels')
    PHR_LABELS = T.ivector(name='phr_labels')


    #load data
    train_set = H5PYDataset(TRAIN_FILE,  which_sets=('train',))
    valid_set = H5PYDataset(VALID_FILE, which_sets=('test',))
    


    spk_network, phr_network = build_rnn(net_input=X, mask_input= MASK)
    
    network_output_spk = lasagne.layers.get_output(spk_network)
    network_output_phr = lasagne.layers.get_output(phr_network)

    val_prediction_spk = lasagne.layers.get_output(spk_network, deterministic=True)
    val_prediction_phr = lasagne.layers.get_output(phr_network, deterministic=True)
    #needed for accuracy
    
    val_acc_spk = T.mean(T.eq(T.argmax(val_prediction_spk, axis=1), SPK_LABELS), dtype=theano.config.floatX)
    val_acc_phr = T.mean(T.eq(T.argmax(val_prediction_phr, axis=1), PHR_LABELS), dtype=theano.config.floatX)

    #training accuracy
    train_acc_spk = T.mean(T.eq(T.argmax(network_output_spk, axis=1), SPK_LABELS), dtype=theano.config.floatX)
    train_acc_phr = T.mean(T.eq(T.argmax(network_output_phr, axis=1), PHR_LABELS), dtype=theano.config.floatX)
    
    #cost function    
    spk_cost = lasagne.objectives.categorical_crossentropy(network_output_spk, SPK_LABELS)
    phr_cost = lasagne.objectives.categorical_crossentropy(network_output_phr, PHR_LABELS)
    total_cost = spk_cost + phr_cost
    mean_cost = total_cost.mean()
    
    #Validation cost
    val_spk_cost = lasagne.objectives.categorical_crossentropy(val_prediction_spk, SPK_LABELS)
    val_phr_cost = lasagne.objectives.categorical_crossentropy(val_prediction_phr, PHR_LABELS)
    val_cost = val_spk_cost + val_phr_cost
    val_mcost = val_cost.mean()

    #Get parameters of both encoder and decoder
    all_parameters = lasagne.layers.get_all_params([network], trainable=True)

    print("Trainable Model Parameters")
    print("-"*40)
    for param in all_parameters:
        print(param, param.get_value().shape)
    print("-"*40)
    #add grad clipping to avoid exploding gradients
    all_grads = [T.clip(g,-3,3) for g in T.grad(mean_cost, all_parameters)]
    all_grads = lasagne.updates.total_norm_constraint(all_grads,3)

    updates = lasagne.updates.adam(all_grads, all_parameters, learning_rate=LEARNING_RATE)

    train_func = theano.function([X, MASK, LABELS], [mean_cost, train_acc], updates=updates)

    val_func = theano.function([X, MASK, LABELS], [val_mcost, val_acc])

    trainerr=[]
    epoch=0 #set the epoch counter
    
    val_prev = np.inf
    a_prev = -np.inf

    print("Starting training...")
        # We iterate over epochs:
    while 'true':
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        tr_acc = 0
        train_batches = 0

        h1=train_set.open()
        h2=valid_set.open()

        scheme = ShuffledScheme(examples=train_set.num_examples, batch_size=64)
        scheme1 = SequentialScheme(examples=valid_set.num_examples, batch_size=128)


        train_stream = DataStream(dataset=train_set, iteration_scheme=scheme)
        valid_stream = DataStream(dataset=valid_set, iteration_scheme=scheme1)

        start_time = time.time()

        for data in train_stream.get_epoch_iterator():
            t_data, t_mask, _, t_labs = data
            terr, tacc = train_func(t_data, t_mask, t_labs)
            train_err += terr
            tr_acc += tacc
            train_batches += 1

        val_err = 0
        val_acc = 0
        val_batches = 0

        for data in valid_stream.get_epoch_iterator():
            v_data, v_mask, _, v_tars = data
            err, acc = val_func(v_data, v_mask ,v_tars)
            val_err += err
            val_acc += acc
            val_batches += 1

        trainerr.append(train_err/train_batches)

        epoch+=1
        train_set.close(h1)
        valid_set.close(h2)
        
        #Display
        if display:
            
            print("Epoch {} of {} took {:.3f}s ".format(
            epoch, NUM_EPOCHS, time.time() - start_time))
            print("  training loss:\t\t{:.6f} ".format(train_err / train_batches))
            print("  training accuracy:\t\t{:.2f} % ".format(
                    tr_acc / train_batches * 100))
            print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
            print("  validation accuracy:\t\t{:.2f} % ".format(
                    val_acc / val_batches * 100))

        logfile = os.path.join(SAVE_PATH,MODEL_NAME,'logs',MODEL_NAME+'.log')
        flog1 = open(logfile,'ab')

        flog1.write("Epoch {} of {} took {:.3f}s\n ".format(
        epoch, NUM_EPOCHS, time.time() - start_time))
        flog1.write("  training loss:\t\t{:.6f} ".format(train_err / train_batches))
        flog1.write("  training accuracy:\t\t{:.2f} %\n ".format(
            tr_acc / train_batches * 100))
        flog1.write("  validation loss:\t\t{:.6f}\n".format(val_err / val_batches))
        flog1.write("  validation accuracy:\t\t{:.2f} %\n ".format(
            val_acc / val_batches * 100))
        flog1.write("\n")
        flog1.close()

        if epoch == NUM_EPOCHS:
            break

        valE = val_err/val_batches
        valA = val_acc / val_batches

        #save the max accuracy model
        max_val = a_prev

        #save model with highest accuracy
        if valA > max_val:

            model_params1 = lasagne.layers.get_all_param_values(network)

            model1_name = MODEL_NAME+'_acc' + '.pkl'

            vpth1 = os.path.join(SAVE_PATH, MODEL_NAME, 'weights',model1_name)

            fsave = open(vpth1,'wb')  

            cPickle.dump(model_params1, fsave, protocol=cPickle.HIGHEST_PROTOCOL)

            fsave.close()

            max_val = valA

        #Patience

        if valE > val_prev:
            c+=1

            #save the model incase
            model_params1 = lasagne.layers.get_all_param_values(network)

            model1_name = MODEL_NAME + '_ofit'  + '.pkl'

            vpth1 = os.path.join(SAVE_PATH, MODEL_NAME, 'weights',model1_name)

            fsave = open(vpth1,'wb')  

            cPickle.dump(model_params1, fsave, protocol=cPickle.HIGHEST_PROTOCOL)

            fsave.close()

            val_prev=valE

        else:
            c=0
            val_prev=valE

        if c==5:
            break

    #Save the final model

    print('Saving Model ...')
    model_params = lasagne.layers.get_all_param_values(network)
    model1_name = MODEL_NAME+'_final' + '.pkl'
    vpth = os.path.join(SAVE_PATH, MODEL_NAME,'weights',model1_name)
    fsave = open(vpth,'wb')  
    cPickle.dump(model_params, fsave, protocol=cPickle.HIGHEST_PROTOCOL)
    fsave.close()
    
if __name__ == '__main__':
    main()


In [12]:
#Save the final model
spth = '/misc/data15/reco/bhattgau/Rnn/Projects/Rvector/Weights/basic-softmax'

tfile = open('/misc/data15/reco/bhattgau/Rnn/Projects/Rvector/Weights/basic-softmax/trainerr.npy','wb')
trainerr = np.asarray(trainerr, dtype='float32')
np.save(tfile,trainerr)

print('Saving Model ...')
model_params = lasagne.layers.get_all_param_values(l_softmax)
model_name = 'basic_softmax_500' + '.pkl'
vpth = os.path.join(spth, model_name)
fsave = open(vpth,'wb')  
cPickle.dump(model_params, fsave, protocol=cPickle.HIGHEST_PROTOCOL)
fsave.close()

Writing spk_softmax_fuel.py


In [18]:
%%writefile /misc/data15/reco/bhattgau/Rnn/code/Code_/frame_softmax.py

import htkmfc
import os
import lasagne
import time
import theano
import theano.tensor as T
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import cPickle
import sys

from fuel.datasets import IndexableDataset
from fuel.schemes import ShuffledScheme, SequentialScheme
from fuel.streams import DataStream

from collections import OrderedDict
import h5py

from fuel.datasets import H5PYDataset
from fuel.converters.base import fill_hdf5_file
from utils import load_data

cFile = sys.argv[1]

f=open(cFile)
lines = f.readlines()
lines = [l.strip() for l in lines]
values = [v.split()[1] for v in lines]


N_HIDDEN = int(values[0])
F_DIM = int(values[1])
LEARNING_RATE = np.cast['float32'](values[2])
NUM_EPOCHS = int(values[3])
NUM_SPEAKERS = int(values[4])
MODEL_NAME = values[5]
SAVE_PATH = values[6]
#data
TRAIN_FILE = values[7]
VALID_FILE = values[8]


# Min/max sequence length
MAX_LENGTH = 800

def build_rnn(net_input=None, mask_input=None):

    print("Building network ...")

    l_in = lasagne.layers.InputLayer(shape=(None, MAX_LENGTH, F_DIM), input_var = net_input)
    n_batch,_,_ = l_in.input_var.shape
    #print lasagne.layers.get_output(l_in, inputs={l_in: X}).eval({X: x_dummy}).shape

    l_mask = lasagne.layers.InputLayer(shape=(None, MAX_LENGTH), input_var = mask_input)
    #print lasagne.layers.get_output(l_mask, inputs={l_mask: Mask}).eval({Mask: mask}).shape

    #initialize the gates
    l_forward = lasagne.layers.GRULayer(l_in, N_HIDDEN, precompute_input=True, mask_input=l_mask)
    l_backward = lasagne.layers.GRULayer(l_in, N_HIDDEN, precompute_input=True, mask_input=l_mask, backwards=True)
    l_sum = lasagne.layers.ElemwiseSumLayer([l_forward, l_backward])

    l_reshape1 = lasagne.layers.ReshapeLayer(l_sum, (-1, N_HIDDEN))

    l_softmax = lasagne.layers.DenseLayer(l_reshape1, num_units=NUM_SPEAKERS, nonlinearity=lasagne.nonlinearities.softmax)
    
    return l_softmax

    

def main():
        
    X = T.tensor3(name='input',dtype='float32')
    MASK = T.matrix(name = 'mask', dtype='float32')
    LABELS = T.ivector(name='labels')
    
    #load data
    train_set = H5PYDataset(TRAIN_FILE,  which_sets=('train',))
    valid_set = H5PYDataset(VALID_FILE, which_sets=('test',))
    
    network = build_rnn(net_input=X, mask_input= MASK)
    
    network_output = lasagne.layers.get_output(network)

    val_prediction = lasagne.layers.get_output(network, deterministic=True)
    #needed for accuracy
    val_acc = T.mean(T.eq(T.argmax(val_prediction, axis=1), LABELS), dtype=theano.config.floatX)
    #training accuracy
    train_acc = T.mean(T.eq(T.argmax(network_output, axis=1), LABELS), dtype=theano.config.floatX)

    #T.argmax(network_output, axis=1).eval({X: d1, Mask: m1})

    #print network_output.eval({X: d1, Mask: m1})[1][97]
    #cost function
    total_cost = lasagne.objectives.categorical_crossentropy(network_output, LABELS)
    #total_cost = -(labels*T.log(network_output) + (1-labels)*T.log(1-network_output)) 
    masked_cost = total_cost*MASK.flatten()
    mean_cost = total_cost.mean()
    #accuracy function
    val_cost = lasagne.objectives.categorical_crossentropy(val_prediction, LABELS)
    val_cost = val_cost*MASK.flatten()
    val_mcost = val_cost.mean()

    #Get parameters of both encoder and decoder
    all_parameters = lasagne.layers.get_all_params([network], trainable=True)

    print("Trainable Model Parameters")
    print("-"*40)
    for param in all_parameters:
        print(param, param.get_value().shape)
    print("-"*40)
    #add grad clipping to avoid exploding gradients
    all_grads = [T.clip(g,-3,3) for g in T.grad(mean_cost, all_parameters)]
    all_grads = lasagne.updates.total_norm_constraint(all_grads,3)

    updates = lasagne.updates.adam(all_grads, all_parameters, learning_rate=LEARNING_RATE)

    train_func = theano.function([X, MASK, LABELS], [mean_cost, train_acc], updates=updates)

    val_func = theano.function([X, MASK, LABELS], [val_mcost, val_acc])

    trainerr=[]
    epoch=0 #set the epoch counter
    
    val_prev = np.inf
    a_prev = -np.inf

    print("Starting training...")
        # We iterate over epochs:
    while 'true':
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        tr_acc = 0
        train_batches = 0

        h1=train_set.open()
        h2=valid_set.open()

        scheme = ShuffledScheme(examples=train_set.num_examples, batch_size=64)
        scheme1 = SequentialScheme(examples=valid_set.num_examples, batch_size=64)


        train_stream = DataStream(dataset=train_set, iteration_scheme=scheme)
        valid_stream = DataStream(dataset=valid_set, iteration_scheme=scheme1)

        start_time = time.time()

        for data in train_stream.get_epoch_iterator():
            t_data, t_mask,t_labs = data
            t_labs = t_labs.flatten()
            terr, tacc = train_func(t_data, t_mask, t_labs)
            train_err += terr
            tr_acc += tacc
            train_batches += 1

        val_err = 0
        val_acc = 0
        val_batches = 0

        for data in valid_stream.get_epoch_iterator():
            v_data, v_mask, v_tars = data
            v_tars = v_tars.flatten()
            
            err, acc = val_func(v_data, v_mask ,v_tars)
            val_err += err
            val_acc += acc
            val_batches += 1

        trainerr.append(train_err/train_batches)

        epoch+=1
        train_set.close(h1)
        valid_set.close(h2)
        
        #Display
        if display:
            
            print("Epoch {} of {} took {:.3f}s ".format(
            epoch, NUM_EPOCHS, time.time() - start_time))
            print("  training loss:\t\t{:.6f} ".format(train_err / train_batches))
            print("  training accuracy:\t\t{:.2f} % ".format(
                    tr_acc / train_batches * 100))
            print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
            print("  validation accuracy:\t\t{:.2f} % ".format(
                    val_acc / val_batches * 100))

        logfile = os.path.join(SAVE_PATH,MODEL_NAME,'logs',MODEL_NAME+'.log')
        flog1 = open(logfile,'ab')

        flog1.write("Epoch {} of {} took {:.3f}s\n ".format(
        epoch, NUM_EPOCHS, time.time() - start_time))
        flog1.write("  training loss:\t\t{:.6f} ".format(train_err / train_batches))
        flog1.write("  training accuracy:\t\t{:.2f} %\n ".format(
            tr_acc / train_batches * 100))
        flog1.write("  validation loss:\t\t{:.6f}\n".format(val_err / val_batches))
        flog1.write("  validation accuracy:\t\t{:.2f} %\n ".format(
            val_acc / val_batches * 100))
        flog1.write("\n")
        flog1.close()

        if epoch == NUM_EPOCHS:
            break

        valE = val_err/val_batches
        valA = val_acc / val_batches

        #save the max accuracy model
        max_val = a_prev

        #save model with highest accuracy
        if valA > max_val:

            model_params1 = lasagne.layers.get_all_param_values(network)

            model1_name = MODEL_NAME+'_acc' + '.pkl'

            vpth1 = os.path.join(SAVE_PATH, MODEL_NAME, 'weights',model1_name)

            fsave = open(vpth1,'wb')  

            cPickle.dump(model_params1, fsave, protocol=cPickle.HIGHEST_PROTOCOL)

            fsave.close()

            max_val = valA

        #Patience

        if valE > val_prev:
            c+=1

            #save the model incase
            model_params1 = lasagne.layers.get_all_param_values(network)

            model1_name = MODEL_NAME + '_ofit'  + '.pkl'

            vpth1 = os.path.join(SAVE_PATH, MODEL_NAME, 'weights',model1_name)

            fsave = open(vpth1,'wb')  

            cPickle.dump(model_params1, fsave, protocol=cPickle.HIGHEST_PROTOCOL)

            fsave.close()

            val_prev=valE

        else:
            c=0
            val_prev=valE

        if c==5:
            break

    #Save the final model

    print('Saving Model ...')
    model_params = lasagne.layers.get_all_param_values(network)
    model1_name = MODEL_NAME+'_final' + '.pkl'
    vpth = os.path.join(SAVE_PATH, MODEL_NAME,'weights',model1_name)
    fsave = open(vpth,'wb')  
    cPickle.dump(model_params, fsave, protocol=cPickle.HIGHEST_PROTOCOL)
    fsave.close()
    
if __name__ == '__main__':
    main()


Overwriting /misc/data15/reco/bhattgau/Rnn/code/Code_/frame_softmax.py
